In [47]:
import pandas as pd
import numpy as np

df_ds = pd.read_csv('titanic.csv')
df=df_ds.copy()
df = df.drop(df.columns[[10]],axis = 1)
df = df.dropna(axis=0)


_ft = input("Enter the Selected Feature Name: ")
print("Select the Feature Type: \n Nominal: Press 1 \n Binary: Press 2 \n Numeric: Press 3 \n Unknown: Press 4")
_type = input("Enter your answer: ")

if _type==4 and (df_ds[_ft].dtype==np.float64 or df_ds[_ft]==np.int64):
    _type=3

loc=df.columns.get_loc(_ft)

X = df.iloc[:,[2,9]].values
y = df.iloc[:,loc].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

if _type=='1':
    import iMICE
    
    def processAge(x):
        if x >= 0 and x < 2:
            return 0
        elif x >= 2 and x <= 12:
            return 1
        elif x >= 13 and x <= 59:
            return 2
        elif x >= 60:
            return 3
    
    df_ds['AgeCode'] = df_ds['Age'].map(processAge)
    
    df_2 = df_ds[['AgeCode', 'Pclass', 'Fare']].copy()
    
    mice_result = iMICE.iMICE(verbose=False, init_fill_method="median", impute_type="pmm", n_imputations=7).complete(np.matrix(df_2))
    
    df_3 = pd.DataFrame(mice_result, columns = ['AgeCode', 'Pclass', 'Fare'])
    df_ds['New'] = df_3['AgeCode'].copy()
    

#Binary
elif _type=='2':
    import iMICE
    
    df_ds['AgeBinary'] = df['Age'].apply(lambda x: 1 if x >= 19 else 0)
    df_2 = df_ds[['AgeBinary', 'Pclass', 'Fare']].copy()
    
    mice_result = iMICE.iMICE(verbose=False, init_fill_method="median", impute_type="pmm", n_imputations=7).complete(np.matrix(df_2))
    
    df_3 = pd.DataFrame(mice_result, columns = ['AgeBinary', 'Pclass', 'Fare'])
    df_ds['New'] = df_3['AgeBinary'].copy()
    

#Numeric
else:
    from sklearn.ensemble import RandomForestRegressor

    regressor = RandomForestRegressor(n_estimators=20, random_state=0)  
    regressor.fit(X_train, y_train)
    
    X_test_1=df_ds.iloc[:,[2,9]].values.copy()
    
    y_pred = regressor.predict(X_test_1)
    
    df_ds['RealAge'] = df_ds['Age'].copy()
    mf=df_ds['Age'].isnull()
    df_ds['New']=df_ds['Age'].copy()
    #df_ds['New']=df_ds['Age'].copy()
    df_ds['newAge'] = y_pred
    m=df_ds['New'].copy()
    m[mf]=df_ds['newAge']
    df_ds['New']=m
    df_ds = df_ds.drop(df_ds.columns[[14]],axis = 1)
    
    def AgeCode(x):
        if x >= 0 and x <= 9:
            return 0
        elif x >= 10 and x <= 19:
            return 1
        elif x >= 20 and x <= 29:
            return 2
        elif x >= 30 and x <= 39:
            return 3
        elif x >= 40 and x <= 49:
            return 4
        elif x >= 50 and x <= 59:
            return 5
        elif x >= 60 and x <= 69:
            return 6
        elif x >= 70 and x <= 79:
            return 7
        elif x >= 80 and x <= 89:
            return 8
        elif x >= 90 and x <= 99:
            return 9
    df_ds['Age'] = df_ds['New'].map(AgeCode).copy()
    #df_ds['Age'] = df_ds['Age'].map(lambda x: int(x))
    
#Name Value
salutation = ["a v m", "admiraal", "air cdre", "air commodore", "air marshal", "air vice marshal", "alderman", "alhaji", "ambassador", "baron", "barones", "brig", "brig gen", "brig general", "brigadier", "brigadier general", "brother", "canon", "capt", "captain", "cardinal", "cdr", "chief", "cik", "cmdr", "col", "col dr", "colonel", "commandant", "commander", "commissioner", "commodore", "comte", "comtessa", "congressman", "conseiller", "consul", "conte", "contessa", "corporal", "councillor", "count", "countess", "crown prince", "crown princess", "dame", "datin", "dato", "datuk", "datuk seri",
            "deacon", "deaconess", "dean", "dhr", "dipl ing", "doctor", "dott", "dott sa", "dr", "dr ing", "dra", "drs", "embajador", "embajadora", "en", "encik", "eng", "eur ing", "exma sra", "exmo sr", "f o", "father", "first lieutient", "first officer", "flt lieut", "flying officer", "fr", "frau", "fraulein", "fru", "gen", "generaal", "general", "governor", "graaf", "gravin", "group captain", "grp capt", "h e dr", "h h", "h m", "h r h", "hajah", "haji", "hajim", "her highness", "her majesty", "herr", "high chief", "his highness",
            "his holiness", "his majesty", "hon", "hr", "hra", "ing", "ir", "jonkheer", "judge", "justice", "khun ying", "kolonel", "lady", "lcda", "lic", "lieut", "lieut cdr", "lieut col", "lieut gen", "lord", "m", "m l", "m r", "madame", "mademoiselle", "maj gen", "major", "master", "mevrouw", "miss", "mlle", "mme", "monsieur", "monsignor", "mr", "mrs", "ms", "mstr", "nti", "pastor", "president", "prince", "princess", "princesse", "prinses", "prof", "prof dr", "prof sir", "professor",
            "puan", "puan sri", "rabbi", "rear admiral", "rev", "rev canon", "rev dr", "rev mother", "reverend", "rva", "senator", "sergeant", "sheikh", "sheikha", "sig", "sig na", "sig ra", "sir", "sister", "sqn ldr", "sr", "sr d", "sra", "srta", "sultan", "tan sri", "tan sri dato", "tengku", "teuku",
            "mrs.", "mr.", "most.", "md.", "master.", "ms.", "alhaz", "hazi", "haji", "mohammad", "muhammad", "dr.", "miss", "prof.", "kazi", "engineer"] 
def processName(name):
        name = name.lower()  # Make name all letter in lowercase 
        namesplits = name.split()  # To break a large string down into smaller
        for sals in salutation:
            if sals in namesplits:
                namesplits.remove(sals) # To remove name salutation and save in namesplits variable
        
        trans = str.maketrans('', '', 'aeiou')  # The method maketrans() returns a translation table that maps each character in the intabstring into the character at the same position in the outtab string. outtab − This is the string having corresponding mapping character.
        for i, nams in enumerate(namesplits):  # enumerate() - It returns an enumerate object.
            namesplits[i] = nams[0] + nams[1:].translate(trans)  # translate() method takes the translation table to replace/translate characters in the given string as per the mapping table.
        
        name = ' '.join(namesplits)  # By join() - simply concatenate two strings
        
        name = name.replace('g', 'j')
        name = name.replace('z', 'j')
        name = name.replace('q', 'k')
        
        trans = str.maketrans('aeioubcdfhjklmnprstvwxy .-', 'tuvwxabcdefghijklmnopqrsyz')  # The string maketrans() method returns a mapping table for translation usable for translate() method.
        name = name.translate(trans)  # The string translate() method returns a string where each character is mapped to its corresponding character in the translation table.
        
        return name
    
#Secure Gender
gendermap = {"male": 0, "female": 1}
df_ds.replace({"Sex": gendermap}, inplace = True)

df_ds['Name'] = df_ds['Name'].map(processName).copy()

secure_ds = df_ds[['PassengerId','Survived','Pclass','Name','Sex','Age','Ticket','Fare','Embarked']].copy()

secure_ds

Enter the Selected Feature Name: Age
Select the Feature Type: 
 Nominal: Press 1 
 Binary: Press 2 
 Numeric: Press 3 
 Unknown: Press 4
Enter your answer: 4


,PassengerId,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Embarked
0,1,0,3,"aljc,swpjsellm",0,2.0,A/5 21171,7.2500,S
1,2,1,1,"bijfm,sfejsalchrs(dhljbsalffmsnerl)",1,3.0,PC 17599,71.2833,C
2,3,1,3,"eggjj,simmyshj",1,2.0,STON/O2. 3101282,7.9250,S
3,4,1,1,"dnlhh,sfbgmsenes(hhrsirskh)",1,3.0,113803,53.1000,S
4,5,0,3,"thhj,sphhisejlr",0,3.0,373450,8.0500,S
5,6,0,3,"ilj,sfim",0,1.0,330877,8.4583,Q
6,7,0,1,"ibblner,sninersf",0,5.0,17463,51.8625,S
7,8,0,3,"khmmj,sfmnshjlc",0,0.0,349909,21.0750,S
8,9,1,3,"fejmj,swmblsps(hmanesohehijsalf)",1,2.0,347742,11.1333,S
9,10,1,2,"jmml,sjbehms(chstbei)",1,1.0,237736,30.0708,C


In [31]:
secure_ds_2 = df_ds[['PassengerId','Survived','Pclass']].copy()

In [20]:
import pandas as pd
import numpy as np

df_ds_1 = pd.read_csv('titanic.csv')

In [43]:
secure_ds_2['Ticket']= df_ds[['Ticket']].copy()

In [48]:
secure_ds.head(18)

,PassengerId,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Embarked
0,1,0,3,"aljc,swpjsellm",0,2.0,A/5 21171,7.2500,S
1,2,1,1,"bijfm,sfejsalchrs(dhljbsalffmsnerl)",1,3.0,PC 17599,71.2833,C
2,3,1,3,"eggjj,simmyshj",1,2.0,STON/O2. 3101282,7.9250,S
3,4,1,1,"dnlhh,sfbgmsenes(hhrsirskh)",1,3.0,113803,53.1000,S
4,5,0,3,"thhj,sphhisejlr",0,3.0,373450,8.0500,S
5,6,0,3,"ilj,sfim",0,1.0,330877,8.4583,Q
6,7,0,1,"ibblner,sninersf",0,5.0,17463,51.8625,S
7,8,0,3,"khmmj,sfmnshjlc",0,0.0,349909,21.0750,S
8,9,1,3,"fejmj,swmblsps(hmanesohehijsalf)",1,2.0,347742,11.1333,S
9,10,1,2,"jmml,sjbehms(chstbei)",1,1.0,237736,30.0708,C
